In [16]:
!python --version

Python 3.7.13


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Have sensor data be incrementally stored in individual files for every time that the file storage maximum (e.g., 30 MB) is reached

!pip install boto3 botocore bluepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import socket
import boto3
import pandas as pd
from botocore.config import Config

In [24]:
REGION = 'US-WEST-1'
VERSION = 'v4'
ACCESS_KEY = 'AKIAe_Ipl5OhS5alNXWFVOvkYQ'
SECRET_ACCESS_KEY = 'gOaAOWWGN72bjls/lbDGp0+B+MQA9SDAxt1yacMB'
END_POINT_URL = 'https://s3.seagate.com'

s3_resource = boto3.resource('s3', endpoint_url=END_POINT_URL, 
                            aws_access_key_id=ACCESS_KEY,
                            aws_secret_access_key=SECRET_ACCESS_KEY, 
                            config=Config(signature_version=VERSION),
                            region_name=REGION,
                            verify=False)

s3_client = boto3.client('s3', endpoint_url=END_POINT_URL,
                         aws_access_key_id=ACCESS_KEY,
                         aws_secret_access_key=SECRET_ACCESS_KEY,
                         config=Config(signature_version=VERSION),
                         region_name=REGION,
                         verify=False)

In [26]:
s3_client.create_bucket(Bucket="stm32_cortx_bucket")

EndpointConnectionError: ignored

In [ ]:
stm32_tilebox_data_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CORTX Integrations/20220623_014044_Temperature.csv')
s3_client.put_object(Body=stm32_tilebox_data_csv, Bucket="stm32_cortx_bucket", Key="stm32_tilebox_data.csv")

In [ ]:
stm32_tilebox_df = s3_client.download_file('stm32_cortx_bucket', 'stm32_tilebox_data.csv', 's3://stm32_cortx_bucket/stm32_tilebox_data.csv')

# View downloaded .csv file
stm32_tilebox_df

In [21]:
# Need to retrieve data from one of the SensorTile.box's sensors (i.e., temperature, pressure)
# get data measurement units (e.g., time and F/C for temperature) for .csv creation
# Store to CORTX S3 bucket
def create_csv(data): 
  pass

In [ ]:
sensortile_box_name = "TILEBOX"
port = 2
buf_size = 1024

s = socket.socket(socket.AF_BLUETOOTH, socket.SOCK_STREAM, socket.BTPROTO_RFCOMM)
s.bind((sensortile_box_name, port))
s.listen(1)

data = None

try: 
    print('Listening for connection...')
    client, address = s.accept()
    print(f'Connected to {address}')
    
    while True: 
        data = client.recv(buf_size)
        if data: 
            print(data)
except Exception as e: 
    print(f'Something went wrong: {e}')
    client.close()
    s.close()

stm32_tilebox_data_csv = create_csv(data)
s3_client.put_object(Body=stm32_tilebox_data_csv , Bucket="stm32_cortx_bucket", Key="stm32_tilebox_data.csv")